In [1]:
import cv2
import os
import numpy as np
import glob
import math
import scipy
from scipy.spatial import distance
from scipy import signal
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn import metrics

In [2]:
##Lấy danh sách mẫu mống mắt
cwd = os.getcwd()
for dir_path, dir_names, file_names in os.walk(cwd):
    for f in file_names:
        list_file = open("list_file.txt","a+")
        temp=str(os.path.join(dir_path,f)+'\n')
        if '.bmp' in temp:
            list_file.write(temp)
list_file = open("list_file.txt","r")
images_train=[]
images_test=[]
ID_images_train=[]
ID_images_test=[]
if list_file.mode == 'r':
    data =list_file.readlines()
    for i in data[1000:1025]:
        if i[-6:-5]=='1':
            ID_images_test.append(i[-10:-5])
            images_test.append(cv2.imread(i[:-1]))
        else:
            ID_images_train.append(i[-10:-5])
            images_train.append(cv2.imread(i[:-1]))

In [3]:
print(ID_images_test)

['201_1', '202_1', '203_1', '204_1', '205_1']


In [4]:
print(ID_images_train)

['201_2', '201_3', '201_4', '201_5', '202_2', '202_3', '202_4', '202_5', '203_2', '203_3', '203_4', '203_5', '204_2', '204_3', '204_4', '204_5', '205_2', '205_3', '205_4', '205_5']


In [5]:
def ImageEnhancement(normalized):
    enhanced=[]
    for res in normalized:
        res = res.astype(np.uint8)
        im=cv2.equalizeHist(res)
        enhanced.append(im)
    return enhanced

In [6]:
#modulating function as defined in paper
def m(x ,y, f):
    val = np.cos(2*np.pi*f*math.sqrt(x **2 + y**2))
    return val
#spatial filter as defined in paper
def gabor(x, y, dx, dy, f):
    gb = (1/(2*math.pi*dx*dy))*np.exp(-0.5*(x**2 / dx**2 + y**2 / dy**2)) * m(x, y, f)
    return gb

#function to calculate spatial filter over 8x8 blocks
def spatial(f,dx,dy):
    sfilter=np.zeros((8,8))
    for i in range(8):
        for j in range(8):
            sfilter[i,j]=gabor((-4+j),(-4+i),dx,dy,f)
    return sfilter

def get_vec(convolvedtrain1,convolvedtrain2):
    feature_vec=[]
    for i in range(6):
            for j in range(64):
                #Run 8 by 8 filtered block iteratively over the entire image
                start_height = i*8
                end_height = start_height+8
                start_wid = j*8
                end_wid = start_wid+8
                grid1 = convolvedtrain1[start_height:end_height, start_wid:end_wid]
                grid2 = convolvedtrain2[start_height:end_height, start_wid:end_wid]

                # Channel 1
                absolute = np.absolute(grid1)
                # mean
                mean = np.mean(absolute)
                feature_vec.append(mean)
                #deviation
                std = np.mean(np.absolute(absolute-mean))
                feature_vec.append(std)

                # Channel 2
                absolute = np.absolute(grid2)
                # mean
                mean = np.mean(absolute)
                feature_vec.append(mean)
                #deviation
                std = np.mean(np.absolute(absolute-mean))
                feature_vec.append(std)

    return feature_vec

def FeatureExtraction(enhanced):
    con1=[]
    con2=[]
    #get spatial filters
    filter1=spatial(0.67,3,1.5)
    filter2=spatial(0.67,4,1.5) 
    
    feature_vector=[]
    
    for i in range(len(enhanced)):
        img=enhanced[i]
        #define a 48x512 region over which the filters are applied
        img_roi=img[:48,:]
        
        filtered1=scipy.signal.convolve2d(img_roi,filter1,mode='same')
        filtered2=scipy.signal.convolve2d(img_roi,filter2,mode='same')
        
        con1.append(filtered1)
        con2.append(filtered2)
        fv=get_vec(filtered1,filtered2)
        feature_vector.append(fv)
    return feature_vector #each feature vector has a dimension of 1536



In [7]:
images_train=[ImageEnhancement(i) for i in images_train]
train=[FeatureExtraction(i) for i in images_train]

In [8]:
images_test=[ImageEnhancement(i) for i in images_test]
test=[FeatureExtraction(i) for i in images_test]

In [9]:
feature_vector_test=test
feature_vector_train=train

In [10]:
def score(feature_vector_train,feature_vector_test):
    score=0
    for i in range(0,1536):
        if np.isnan(feature_vector_train[i])==True and np.isnan(feature_vector_test[i])==True:
            score+=0
        elif np.isnan(feature_vector_train[i])==True:
            score+=feature_vector_test[i]
        elif np.isnan(feature_vector_test[i])==True:
            score+=feature_vector_train[i]
        else:
            score+=abs(feature_vector_train[i]-feature_vector_test[i])
    return score

In [12]:
a=[[sum([score(feature_vector_train[i][j],feature_vector_test[k][j]) for j in range(0,48)]) for i in range(0,20)] for k in range(0,5)]

In [13]:
a

[[723.7535501160994,
  454.1971814825695,
  545.1467654421451,
  463.4820108041715,
  747.5951368622896,
  879.8224529878074,
  844.3052269933723,
  814.3328080140591,
  852.6587524034929,
  912.4608049707211,
  882.8055361370078,
  929.8900882344667,
  757.8087521342526,
  747.4508177497273,
  714.6661539837762,
  756.7795557354184,
  763.5096479748084,
  745.4578929968806,
  772.3307823684322,
  876.5629056302814],
 [867.296957567554,
  758.9560495313262,
  747.7725624751979,
  763.5462555481927,
  444.54554831003065,
  539.5789003854525,
  484.1461505158555,
  578.1026927618449,
  894.7334080951427,
  901.4157871589881,
  969.916640315615,
  913.4695299912403,
  881.4511773351059,
  893.5353882498807,
  853.4064292139783,
  883.4913992762006,
  784.3852138161789,
  827.3368351858617,
  777.1575501219216,
  774.530197215708],
 [865.3369222340542,
  862.4437945287876,
  827.0976341273592,
  833.0115798349441,
  892.4459176122666,
  1018.7605914095898,
  927.5245210486714,
  930.042514

In [14]:
result=[ID_images_train[i.index(min(i))][0:3] for i in a]

In [15]:
print(result)

['201', '202', '203', '204', '205']


In [16]:
ID_images_test=[i[0:3] for i in ID_images_test ]

In [17]:
print(ID_images_test)

['201', '202', '203', '204', '205']


In [18]:
from scipy.spatial import distance

In [19]:
1-distance.hamming(result,ID_images_test)

1.0